# Reconstruct the SBML from a set of reactions

This is to test the FBA toolchain

In [1]:
import sys
import os
import copy
import PyFBA
import re
import pickle

import inspect
inspect.getfile(PyFBA)

We are using /home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.0-py3.9.egg/PyFBA/Biochemistry/ModelSEEDDatabase for our data


'/home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.5-py3.9.egg/PyFBA/__init__.py'

# Read the Reactions, Enzymes, and Compounds Sets

This parses the ModelSeed data

In [2]:
modelseed = PyFBA.parse.model_seed.parse_model_seed_data('gramnegative')
print(f"There are {len(modelseed.compounds):,} compounds, {len(modelseed.reactions):,} reactions, and {len(modelseed.enzymes):,} enzymes in total")

There are 33,992 compounds, 43,774 reactions, and 9,423 enzymes in total


In [3]:
for c in modelseed.compounds:
    if '_' in c.name:
        print(c)
    elif 'cpd' in c.name:
        print(c)

cpd29756: Cellulose_biomass
cpd16274: M_trehalose_keto_mycolate_phosphate_1_cyclopropanated_rings
cpd29810: L-Methionine_biomass
cpd29827: Ubiquinone_biomass
cpd29882: C00123_biomass
cpd29890: C00079_biomass
cpd29755: CTP_biomass
cpd16273: M_trehalose_methoxy_mycolate_phosphate_1_cyclopropanated_ring
cpd29907: C00075_biomass
cpd16257: M_Butyrate_Coa
cpd29638: an_acid
cpd29683: 12dag3p_BS
cpd29823: Starch_biomass
cpd29937: C00399_biomass
cpd29803: L-Cysteine_biomass
cpd16272: M_trehalose_methoxy_mycolate_phosphate_2_cyclopropanated_rings
cpd16254: M_trehalose_methoxy_mycolate__1_cyclopropanated_ring
cpd29873: Nucleic_acids
cpd16253: M_trehalose_methoxy_mycolate__2_cyclopropanated_rings
cpd16259: M_3_Hydroxybutyryl_CoA
cpd22268: ALPHA_ACETYLGLUCOSAMINIDES
cpd29911: C00041_biomass
cpd29822: Sinapyl alcohol_biomass
cpd29938: C00378_biomass
cpd16089: Phosphatidylleucine_SA
cpd29889: C00135_biomass
cpd16256: M_trehalose_keto_mycolate__2_cyclopropanated_rings
cpd29798: Hexadecanoic acid_bioma

In [4]:
rcpd = set()
for r in modelseed.reactions:
    for c in modelseed.reactions[r].all_compounds():
        if c.id == 'cpd00060':
            rcpd.add(str(c))
print("\n".join(rcpd))

cpd00060: L-Methionine (location: c)
cpd00060: L-Methionine (location: e)


# Create a biomass equation

In [5]:
biomass_equation = PyFBA.metabolism.biomass_equation('gramnegative', modelseed.compounds)

_Remember_: Use `ctrl`+`/` to uncomment a line; `ctrl-A` `ctrl-/` to uncomment this whole block!

In [6]:
# sbml_bm_eqn = '(0.00778132482043096) cpd00063: Ca2 (location: c) +  (0.352889948968272) cpd00156: L-Valine (location: e) +  (0.00778132482043096) cpd00030: Mn2 (location: e) +  (0.00778132482043096) cpd00205: K (location: c) +  (0.428732289454499) cpd00035: L-Alanine (location: e) +  (0.128039715997337) cpd00060: L-Methionine (location: e) +  (0.15480760087483) cpd00066: L-Phenylalanine (location: c) +  (0.00778132482043096) cpd00017: S-Adenosyl-L-methionine (location: c) +  (0.00778132482043096) cpd00010: CoA (location: c) +  (0.0609084652443221) cpd15665: Peptidoglycan-polymer-n-subunits (location: c) +  (0.0841036156544863) cpd00052: CTP (location: c) +  (0.00778132482043096) cpd10516: fe3 (location: e) +  (0.01468498342018) cpd00357: TTP (location: c) +  (0.00778132482043096) cpd00099: Cl- (location: e) +  (0.01468498342018) cpd00356: dCTP (location: c) +  (0.00778132482043096) cpd10515: Fe2 (location: e) +  (0.00778132482043096) cpd00254: Mg (location: c) +  (0.242249358141304) cpd00322: L-Isoleucine (location: e) +  (0.00778132482043096) cpd00058: Cu2 (location: e) +  (0.00778132482043096) cpd00149: Co2 (location: c) +  (0.201205267995816) cpd00041: L-Aspartate (location: e) +  (1) cpd17043: RNA-transcription (location: c) +  (0.219496655995436) cpd00023: L-Glutamate (location: e) +  (0.219496655995436) cpd00053: L-Glutamine (location: e) +  (0.376088782528765) cpd00107: L-Leucine (location: e) +  (0.00778132482043096) cpd00220: Riboflavin (location: e) +  (0.179790960093822) cpd00054: L-Serine (location: e) +  (0.0472899299502361) cpd00065: L-Tryptophan (location: e) +  (0.0609084652443221) cpd02229: Bactoprenyl-diphosphate (location: c) +  (0.00778132482043096) cpd11493: ACP (location: c) +  (1) cpd17041: Protein-biosynthesis (location: c) +  (0.184698405654696) cpd00129: L-Proline (location: e) +  (0.135406821203723) cpd00038: GTP (location: c) +  (0.01468498342018) cpd00241: dGTP (location: c) +  (1) cpd17042: DNA-replication (location: c) +  (0.211466290532188) cpd00161: L-Threonine (location: e) +  (40.1101757365074) cpd00002: ATP (location: c) +  (0.00778132482043096) cpd00016: Pyridoxal-phosphate (location: c) +  (0.00778132482043096) cpd00048: Sulfate (location: e) +  (0.00778132482043096) cpd00003: NAD (location: c) +  (0.01468498342018) cpd00115: dATP (location: c) +  (0.115101904973216) cpd00069: L-Tyrosine (location: e) +  (0.00778132482043096) cpd00015: FAD (location: c) +  (0.201205267995816) cpd00132: L-Asparagine (location: e) +  (0.00778132482043096) cpd00006: NADP (location: c) +  (35.5386858537513) cpd00001: H2O (location: e) +  (0.0762884719008526) cpd00084: L-Cysteine (location: c) +  (0.0794113918032267) cpd00119: L-Histidine (location: e) +  (0.285970236774541) cpd00039: L-Lysine (location: e) +  (0.0908319049068452) cpd00062: UTP (location: c) +  (0.00778132482043096) cpd00034: Zn2 (location: e) +  (0.247156803702178) cpd00051: L-Arginine (location: e) +  (0.510820469745475) cpd00033: Glycine (location: e) >  (40) cpd00008: ADP (location: c) +  (39.9922186751796) cpd00009: Phosphate (location: e) +  (0.00778132482043096) cpd12370: apo-ACP (location: c) +  (1) cpd11416: Biomass (location: c) +  (40) cpd00067: H (location: e) +  (0.0609084652443221) cpd15666: Peptidoglycan-polymer-n-1-subunits (location: c) +  (0.405833094852252) cpd00012: PPi (location: e)'
# sbml_left_compounds = {'cpd00066: L-Phenylalanine (location: c)' : 0.15480760087483,  'cpd00016: Pyridoxal-phosphate (location: c)' : 0.00778132482043096,  'cpd00132: L-Asparagine (location: e)' : 0.201205267995816,  'cpd00156: L-Valine (location: e)' : 0.352889948968272,  'cpd00099: Cl- (location: e)' : 0.00778132482043096,  'cpd00038: GTP (location: c)' : 0.135406821203723,  'cpd00003: NAD (location: c)' : 0.00778132482043096,  'cpd17041: Protein-biosynthesis (location: c)' : 1.0,  'cpd00033: Glycine (location: e)' : 0.510820469745475,  'cpd00322: L-Isoleucine (location: e)' : 0.242249358141304,  'cpd00254: Mg (location: c)' : 0.00778132482043096,  'cpd17043: RNA-transcription (location: c)' : 1.0,  'cpd00048: Sulfate (location: e)' : 0.00778132482043096,  'cpd10515: Fe2 (location: e)' : 0.00778132482043096,  'cpd02229: Bactoprenyl-diphosphate (location: c)' : 0.0609084652443221,  'cpd11493: ACP (location: c)' : 0.00778132482043096,  'cpd00161: L-Threonine (location: e)' : 0.211466290532188,  'cpd00006: NADP (location: c)' : 0.00778132482043096,  'cpd00060: L-Methionine (location: e)' : 0.128039715997337,  'cpd00119: L-Histidine (location: e)' : 0.0794113918032267,  'cpd00052: CTP (location: c)' : 0.0841036156544863,  'cpd00051: L-Arginine (location: e)' : 0.247156803702178,  'cpd15665: Peptidoglycan-polymer-n-subunits (location: c)' : 0.0609084652443221,  'cpd00017: S-Adenosyl-L-methionine (location: c)' : 0.00778132482043096,  'cpd00030: Mn2 (location: e)' : 0.00778132482043096,  'cpd10516: fe3 (location: e)' : 0.00778132482043096,  'cpd00065: L-Tryptophan (location: e)' : 0.0472899299502361,  'cpd00084: L-Cysteine (location: c)' : 0.0762884719008526,  'cpd00023: L-Glutamate (location: e)' : 0.219496655995436,  'cpd17042: DNA-replication (location: c)' : 1.0,  'cpd00356: dCTP (location: c)' : 0.01468498342018,  'cpd00035: L-Alanine (location: e)' : 0.428732289454499,  'cpd00069: L-Tyrosine (location: e)' : 0.115101904973216,  'cpd00220: Riboflavin (location: e)' : 0.00778132482043096,  'cpd00129: L-Proline (location: e)' : 0.184698405654696,  'cpd00357: TTP (location: c)' : 0.01468498342018,  'cpd00205: K (location: c)' : 0.00778132482043096,  'cpd00149: Co2 (location: c)' : 0.00778132482043096,  'cpd00063: Ca2 (location: c)' : 0.00778132482043096,  'cpd00054: L-Serine (location: e)' : 0.179790960093822,  'cpd00001: H2O (location: e)' : 35.5386858537513,  'cpd00010: CoA (location: c)' : 0.00778132482043096,  'cpd00015: FAD (location: c)' : 0.00778132482043096,  'cpd00062: UTP (location: c)' : 0.0908319049068452,  'cpd00107: L-Leucine (location: e)' : 0.376088782528765,  'cpd00241: dGTP (location: c)' : 0.01468498342018,  'cpd00053: L-Glutamine (location: e)' : 0.219496655995436,  'cpd00039: L-Lysine (location: e)' : 0.285970236774541,  'cpd00034: Zn2 (location: e)' : 0.00778132482043096,  'cpd00058: Cu2 (location: e)' : 0.00778132482043096,  'cpd00002: ATP (location: c)' : 40.1101757365074,  'cpd00041: L-Aspartate (location: e)' : 0.201205267995816,  'cpd00115: dATP (location: c)' : 0.01468498342018}
# sbml_right_compounds = {'cpd00067: H (location: e)' : 40.0,  'cpd00012: PPi (location: e)' : 0.405833094852252,  'cpd00008: ADP (location: c)' : 40.0,  'cpd11416: Biomass (location: c)' : 1.0,  'cpd12370: apo-ACP (location: c)' : 0.00778132482043096,  'cpd00009: Phosphate (location: e)' : 39.9922186751796,  'cpd15666: Peptidoglycan-polymer-n-1-subunits (location: c)' : 0.0609084652443221}

# sbml_biomass = PyFBA.metabolism.Reaction('sbml_biomass', 'sbml_biomass')
# sbml_biomass.equation = sbml_bm_eqn
# parsecomp = re.compile('^(cpd\\d+): (.*?) \(location: (.)\)')
# for c in sbml_left_compounds:
#     m = parsecomp.match(c)
#     if not m:
#         sys.stderr.write(f"Can't parse {c}\n")
        
#     cpd = modelseed.get_compound_by_id(m.group(1))
#     if not cpd:
#         cpd = modelseed.get_compound_by_name(m.group(1))    
#     if not cpd:
#         print(f"Left: {m.group(1)} not found")
#         continue

#     if cpd.name != m.group(2):
#         sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
#     newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
#     sbml_biomass.add_left_compounds({newcomp})
#     sbml_biomass.set_left_compound_abundance(newcomp, sbml_left_compounds[c])
        

# for c in sbml_right_compounds:
#     m = parsecomp.match(c)
#     if not m:
#         sys.stderr.write(f"Can't parse {c}\n")

#     cpd = modelseed.get_compound_by_id(m.group(1))
#     if not cpd:
#         cpd = modelseed.get_compound_by_name(m.group(1))    
#     if not cpd:
#         print(f"Left: {m.group(1)} not found")
#         continue
        
#     if cpd.name != m.group(2):
#         sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
#     newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
#     sbml_biomass.add_right_compounds({newcomp})
#     sbml_biomass.set_right_compound_abundance(newcomp, sbml_right_compounds[c])


# Read the media and correct the names

In [7]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = PyFBA.parse.correct_media_names(media, modelseed.compounds)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


We are logging to /home/redwards/GitHubsLinux/PyFBA/iPythonNotebooks/PyFBA.2021-06-09T10:24:34.842424.log
Checking media compounds: Our compounds do not include  Fe2+
Checking media compounds: Our compounds do not include  fe3
Please note: The warnings about media not being found in compounds are not fatal.
It just means that we did not find that compound anywhere in the reactions, and so it is unlikely to be
needed or used. We typically see a few of these in rich media.


# Read the reactions

In [8]:
reactions_to_run = set()
with open('rgood.txt', 'r') as f:
    for l in f:
        if l.startswith('rxn'):
            reactions_to_run.add(l.strip())
print(f"There are {len(reactions_to_run)} reactions to run")

There are 1399 reactions to run


# Add the biomass equation to the reactions to run

In [9]:
#reactions_to_run.add(sbml_biomass)

In [10]:
# for r in reactions_to_run:
#     if r not in modelseed.reactions:
#         sys.stderr.write(f"Error: reaction {r} not found\n")

# Run the FBA

In [11]:
uptake_secretion_reactions = set()

In [12]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix is adding compound cpd00531: Hg2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00311: Guanosine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00220: Riboflavin (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00067: H+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00069: L-Tyrosine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00051: L-Arginine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00034: Zn2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00239: H2S (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00063: Ca2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00226: HYXN (locatio

We are loading 1322 rows and 1592 columns


The FBA completed with a flux value of 4.186715743318518e-13 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1322
Number of reactions in SM: 1592
Revised number of total reactions: 43966
Number of total compounds: 34117
SMat dimensions: 1322 x 1592


## Gapfilling the reactions

In [13]:
for c in biomass_equation.left_compounds:
    print(c)

cpd00220: Riboflavin (location: c)
cpd00015: FAD (location: c)
cpd00034: Zn2+ (location: c)
cpd00069: L-Tyrosine (location: c)
cpd00115: dATP (location: c)
cpd00063: Ca2+ (location: c)
cpd00051: L-Arginine (location: c)
cpd00062: UTP (location: c)
cpd00149: Co2+ (location: c)
cpd00017: S-Adenosyl-L-methionine (location: c)
cpd00060: L-Methionine (location: c)
cpd00119: L-Histidine (location: c)
cpd15665: Peptidoglycan polymer (n subunits) (location: c)
cpd00156: L-Valine (location: c)
cpd00003: NAD (location: c)
cpd00053: L-Glutamine (location: c)
cpd17042: DNA replication (location: c)
cpd00084: L-Cysteine (location: c)
cpd00035: L-Alanine (location: c)
cpd17041: Protein biosynthesis (location: c)
cpd00041: L-Aspartate (location: c)
cpd00010: CoA (location: c)
cpd00002: ATP (location: c)
cpd00357: TTP (location: c)
cpd00099: Cl- (location: c)
cpd00161: L-Threonine (location: c)
cpd00023: L-Glutamate (location: c)
cpd00033: Glycine (location: c)
cpd02229: Bactoprenyl diphosphate (locat

In [14]:
for c in modelseed.compounds:
    if c.name == 'L-Threonine':
        print(c)

cpd00161: L-Threonine (location: e)
cpd00161: L-Threonine
cpd00161: L-Threonine (location: c)


## Comparing to the SBML file that does grow!

We pickled that data, and can load it here. 

In [15]:
sbml_filtered_compounds = pickle.load(open('compounds.pickle', 'rb'))
sbml_reactions = pickle.load(open('reactions.pickle', 'rb'))
sbml_reactions_to_run = pickle.load(open('reactions_to_run.pickle', 'rb'))
sbml_media = pickle.load(open('media.pickle', 'rb'))
sbml_biomass_equation = pickle.load(open('sbml_biomass.pickle', 'rb'))
sbml_uptake_secretion_reactions = pickle.load(open('uptake_secretion_reactions.pickle', 'rb'))

In [16]:
for r in sbml_reactions:
    for c in sbml_reactions[r].all_compounds():
        if '_' in c.name:
            print(c.name)

L_Alanine
L_Methionine
met_L_ala_L
L_Threonine
ala_L_Thr_L
L_Alanine
gly_asp_L
L_Aspartate
Gly_Cys
L_Cysteine
ala_L_glu_L
L_Alanine
L_Glutamate
gly_asn_L
L_Asparagine
L_Leucine
Gly_Leu
L_Alanine
L_alanylglycine
ala_L_asp_L
L_Aspartate
L_Alanine
L_Cysteine
Cys_Gly
Gly_Gln
L_Glutamine
Gly_Phe
L_Phenylalanine
L_Histidine
L_Alanine
Ala_His
gly_glu_L
L_Glutamate
L_Glutamine
L_Alanine
Ala_Gln
L_Tyrosine
Gly_Tyr
L_Proline
gly_pro_L
L_Leucine
L_Alanine
Ala_Leu
Gly_Met
L_Methionine
Glucose_1_phosphate
Butyryl_CoA
Crotonyl_CoA
Methacrylyl_CoA
Isobutyryl_CoA
Tiglyl_CoA
2_Methylbutyryl_CoA
N_Acetyl_D_glucosamine1_phosphate
D_Glucosamine1_phosphate
Acetyl_CoA
N_Acetyl_D_glucosamine1_phosphate
UDP_N_acetylglucosamine
L_Arginine
L_Argininosuccinate
4_Carboxymuconolactone
3_oxoadipate_enol_lactone
2_Demethylmenaquinol_8
2_Demethylmenaquinone_8
Ubiquinol_8
Ubiquinone_8
Menaquinol_8
Menaquinone_8
1_anteisoheptadecanoyl_sn_glycerol_3_phosphate
1_2_dianteisoheptadecanoyl_sn_glycerol_3_phosphate
1_2_diisoh

S_Adenosyl_L_methionine
2_Octaprenyl_3_methyl_6_methoxy_1_4_benzoquinone
2_Octaprenyl_6_methoxy_1_4_benzoquinone
S_Adenosyl_homocysteine
S_Adenosyl_L_methionine
2_Demethylmenaquinone_8
Menaquinone_8
Protein_biosynthesis
5_Methyltetrahydrofolate
5_10_Methylenetetrahydrofolate
5_Methyltetrahydrofolate
5_10_Methylenetetrahydrofolate
UDP_MurNAc
UDP_N_acetylmuramoyl_L_alanine
L_Alanine
Lipid_X
UDP_2_3_bis3_hydroxytetradecanoylglucosamine
Lipid_A_disaccharide
Glyceraldehyde3_phosphate
Indoleglycerol_phosphate
Indoleglycerol_phosphate
L_Serine
Glyceraldehyde3_phosphate
L_Tryptophan
L_Tryptophan
L_Serine
5_Amino_6__5_phosphoribitylaminouracil
5_Amino_6__5_phosphoribosylaminouracil
L_Ribulose
L_Arabinose
D_glucose_6_phosphate
L_Fucose_1_phosphate
D_mannose_6_phosphate
1_2_Propanediol
2_3_Dihydroxy_isovalerate
3_Methyl_2_oxobutanoate
2_3_Dihydroxy_3_methylvalerate
L_Threonine
O_Phospho_L_homoserine
4_Hydroxy_L_threonine
4_Phosphonooxy_threonine
dTDP_4_oxo_6_deoxy_D_glucose
L_Glutamine
D_Glucosam

So if we use the `sbml_reactions` and the `sbml_biomass_equation`, we get growth. Without those reactions we get an error. With the normal biomass equation, we don't get any growth

In [17]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_reactions,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix added compound cpd00107: L_Leucine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00053: L_Glutamine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00060: L_Methionine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00051: L_Arginine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00099: Cl_ (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00205: K (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd00017: S_Adenosyl_L_methionine (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd00156: L_Valine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00069: L_Tyrosine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00058: Cu2 

The FBA completed with a flux value of 400.4414741729352 --> growth: True


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1187
Number of reactions in SM: 1608
Revised number of total reactions: 1782
Number of total compounds: 34156
SMat dimensions: 1187 x 1608


In [18]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        print(c)

cpd00060: L_Methionine (location: e)
cpd00060: L-Methionine (location: e)
cpd00060: L-Methionine (location: c)
cpd00060: L-Methionine


### Note the normal biomass equation does not work!

We need to figure out what is missing!

In [19]:
biomass_equation = PyFBA.metabolism.biomass_equation('gram_negative')
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_reactions,
                                          reactions_to_run, media, biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

AttributeError: 'function' object has no attribute 'get_compound_by_name'

### And the modelseed.reactions gives us an error (or no growth

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in modelseed.reactions:
        print(r)

In [ ]:
tmpr = copy.deepcopy(modelseed.reactions)
tmpr['biomass_equation'] = sbml_reactions['biomass_equation']

In [ ]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, tmpr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
sbml_reactions['EX_cpd01022'].__dict__

In [ ]:
external = set()
for r in reactions_to_run:
    for c in modelseed.reactions[r].all_compounds():
        if c.location == 'e':
            external.add(c)
print(f"There are {len(external)} external compounds")

In [ ]:
newr = {}
for e in external:
    n = PyFBA.metabolism.Reaction(f"EX_{e.id}", readable_name='external rctn {e.id}',
                                 equation = f"(1) {e} = (1) {e}", direction='=')
    n.add_left_compounds(c)
    n.add_right_compounds(c)
    n.set_left_compound_abundance(c, 1)
    n.set_right_compound_abundance(c, 1)
    n.lower_bound = -1000
    n.upper_bound = 1000
    newr[f"EX_{e.id}"] = n


In [ ]:
mr = copy.deepcopy(modelseed.reactions)
mr.update(newr)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
mr['biomass_equation'] = sbml_reactions['biomass_equation']
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r not in mr:
        print(r)

In [ ]:
sbml_reactions['EX_cpd08636'].__dict__

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
del sbml_partial['EX_cpd11416']

In [ ]:
for f in ['EX_cpd15302', 'EX_cpd08636', 'EX_cpd02701']:
    del sbml_partial[f]

In [ ]:
for r in sbml_partial:
    if r not in mr:
        print(r)

In [ ]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

In [ ]:
for r in sbml_reactions:
    if r in modelseed.reactions:
        left_diff = 0
        right_diff = 0
        for l in modelseed.reactions[r].left_compounds:
            try:
                a = sbml_reactions[r].get_left_compound_abundance(l)
            except KeyError:
                left_diff += 1
        for l in modelseed.reactions[r].right_compounds:
            try:
                a = sbml_reactions[r].get_right_compound_abundance(l)
            except KeyError:
                right_diff += 1
        if left_diff > 0 or right_diff > 0:
            print(f"{r}\t{left_diff}\t{right_diff}")

In [ ]:
for r in sbml_reactions:
    print(r)

In [ ]:
sbml_reactions['rxn01516'].__dict__

In [ ]:
modelseed.reactions['rxn01516'].__dict__

In [ ]:
for r in reactions_to_run:
    for c in modelseed.reactions[r].left_compounds:
        print(f"left {c.id} abundance {0 - modelseed.reactions[r].get_left_compound_abundance(c)}")


In [ ]:
modelseed.get_compound_by_id('cpd00037').name

In [ ]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        if isinstance(c, PyFBA.metabolism.CompoundWithLocation):
            print(f"{c} ({c.id}) ({c.name}) ({c.location}): {c.__hash__()}")
            print(f"{hash((c.id, c.name, c.location))}")
        else:
            print(f"{c} ({c.id}) ({c.name}): {c.__hash__()}")

In [ ]:
cid = 'cpd00060'
cname = 'L_Methionine'
cname2 = 'L-Methionine'
cloc = 'e'
for i in range(5):
    print(f"{hash((cid, cname, cloc))}")
    print(f"{hash((cid, cname2, cloc))}")